In [1]:
#!pip install swifter --user
#!pip install numpy --user
#!pip install nltk --user
#!pip install sklearn --user
#!pip install arrow --user

In [2]:
# импорт библиотек
import json 
import numpy as np
import pandas as pd
import re
import string
import nltk
import csv
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer 
import gc
import swifter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle 
import sys
import arrow
import numpy as np
from numpy.linalg import norm
from multiprocessing import Pool
from correct import correct_numbers

In [3]:
# загрузка данных nltk 
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kristina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kristina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
STOP_WORDS = stopwords.words('english') # стоп-слова
lemmatizer = WordNetLemmatizer() # лемматизатор
tokenizer = RegexpTokenizer(r'\w+') # токенизатор (лексер)

In [5]:
# данные о предприятиях
df_business = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [6]:
# убираем предприятия, в которых отсутствуют данные и оставляем только заведения общественного питания (рестораны)
#df_business = df_business.dropna() 
df_business = df_business[df_business['categories'].notna() & df_business['state'].notna()]
df_business_restaurants = df_business[df_business['categories'].str.contains('Restaurants') & df_business['state'].str.contains('NC|SC')] 

# удаляем df_business и вызываем сборщик мусора
del(df_business)
gc.collect()

# оставляем только открытые заведения (1 = open , 0 = closed)
df_business_restaurants = df_business_restaurants[df_business_restaurants['is_open'] == 1]
df_business_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
136,3C0bnFhjkgYP9mWORKg6cA,Chili Man,S Tryon,Charlotte,NC,28202,35.226911,-80.843823,5.0,172,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsDel...","Hot Dogs, Food, Street Vendors, Restaurants","{'Tuesday': '11:0-14:30', 'Wednesday': '11:0-1..."
155,e1tqT4QNOUqDiOvmaIcxfQ,Chef Alyssa's Kitchen,"4001-C Yancey Rd, Ste 100",Charlotte,NC,28217,35.186099,-80.878288,4.5,58,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Active Life, Summer Camps, Personal Chefs, Ven...","{'Monday': '0:0-0:0', 'Tuesday': '18:0-21:0', ..."
176,PlP0c_u3g-3LiWHpb2XnOQ,Lee Cafe,"1046 Regent Pkwy, Ste 107",Fort Mill,SC,29715,35.075476,-80.933904,4.0,267,1,"{'NoiseLevel': 'u'average'', 'RestaurantsReser...","Restaurants, Sushi Bars, Chinese","{'Monday': '17:0-21:30', 'Tuesday': '17:0-21:3..."
197,8k8cMmMcuj5h85mMLMoj2w,RayNathan's,4571 S New Hope Rd,Gastonia,NC,28056,35.207800,-81.074761,4.0,58,1,"{'GoodForKids': 'True', 'WheelchairAccessible'...","Barbeque, Southern, Restaurants, American (Tra...","{'Thursday': '11:0-21:0', 'Friday': '11:0-21:0..."
250,ypILNgy7QFskKAdcPKB2RQ,KFC,1667 Ebenezer Rd,Rock Hill,SC,29732,34.956131,-81.046257,2.0,5,1,"{'GoodForKids': 'True', 'WiFi': ''free'', 'Res...","Restaurants, Fast Food, Chicken Shop, Chicken ...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-22:0', ..."


In [7]:
print(df_business_restaurants.shape)

(3613, 14)


In [8]:
# файл формата json с отзывами очень большой, необходимо преобразовать его в csv 
keys = ["review_id", "user_id", "business_id", "stars", "text"]
with open("review_data.csv", "w",  encoding='utf-8') as fcsv:
    wr = csv.writer(fcsv)
    wr.writerow(['review_id', 'user_id', 'business_id', 'stars', 'text'])
    with open('yelp_dataset/yelp_academic_dataset_review.json', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            wr.writerow([data[keys[0]], data[keys[1]], data[keys[2]], data[keys[3]], data[keys[4]]])

In [8]:
# обработка текста
def text_preprocess(text):
    text = tokenizer.tokenize(text.lower()) # приводим текст в нижний регистр и разбиваем на токены по пробелам и знакам пунктуации
    text = [re.sub('[^a-z\s]', '', w) for w in text]# избавляемся от цифр и нелатинских символов 
    text = [lemmatizer.lemmatize(w) for w in text if w not in STOP_WORDS]# производим лемматизацию и избавляемся от стоп-слов
    return ' '.join(text)

In [10]:
# данные об отзывах
df_review = pd.read_csv("review_data.csv")
# размер датафрейма
print(df_review.shape)
df_review.head()

(8021122, 5)


,review_id,user_id,business_id,stars,text
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2.0,"As someone who has worked with many museums, I..."
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1.0,I am actually horrified this place is still in...
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0,I love Deagan's. I do. I really do. The atmosp...
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1.0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0,"Oh happy day, finally have a Canes near my cas..."


In [11]:
# оставляем только те отзывы, которые относятся к заведениям питания из датафрейма df_business_restaurants
df_review = df_review[df_review['business_id'].isin(df_business_restaurants['business_id'])]
# заменяем отсутствующие отзывы на пустую строку 
df_review["text"].fillna('', inplace=True)

In [13]:
# эффективно обрабатываем текст с помощью swifter и очищаем полученный датафрейм от отзывов с пустым текстом
df_review['text'] = df_review['text'].swifter.apply(text_preprocess)
df_review.shape # итоговый размер датафрейма

(305444, 5)

In [14]:
# записываем полученный файл без индексов
df_review.to_csv('review_data_state.csv', index=False)

In [9]:
class MatrixFactorization(object):
    
    def __init__(self, U, I, learning_rate=0.1, lamda=0.1, num_epoches=5, num_batches=100):
        self.learning_rate = learning_rate # скорость обучения (learning rate)
        self.lamda = lamda # лямбда для L2 регуляризации 
        self.num_epoches = num_epoches  # число эпох
        self.num_batches = num_batches  #число батчей
        self.U = U # users feature matrix (матрица признаков пользователей)
        self.I = I # items feature matrix (матрица признаков заведений)
        self.e = 0 # текущая эпоха

    # функция потерь
    def loss(self, ratings):
        errors = [(rat_ij - np.dot(self.U[i], self.I[j].T))**2 + self.lamda * \
              (norm(self.U[i]) + norm(self.I[j])) for i, j, rat_ij in ratings]
        return sum(errors)

    # обновление матриц U и I с помощью стохастического градиентного спуска
    def update(self, ratings):
        for i, j, rat_ij in ratings:
            rat_ij_hat = np.dot(self.U[i], self.I[j].T)
            grad_U_i = (rat_ij_hat - rat_ij) * self.I[j] + self.lamda * self.U[i]
            grad_I_j = (rat_ij_hat - rat_ij) * self.U[i] + self.lamda * self.I[j]
            self.U[i] -= self.learning_rate * grad_U_i
            self.I[j] -= self.learning_rate * grad_I_j
    
    # настройка модели 
    def fit(self, train, test):
        num_trains = train.shape[0] # число тренировочных данных
        num_tests  = test.shape[0] # число тестовых данных
        while self.e < self.num_epoches:
            self.e += 1
            # перемешиваем тренировочные данные
            mixed_order = np.arange(num_trains)
            np.random.shuffle(mixed_order)
            avg_train_loss = [] # ошибка тренировочных данных 
            avg_test_loss  = [] # ошибка тестовых данных
            batch_size = int(num_trains / self.num_batches) # размер батча
            for batch in range(self.num_batches):
                batch_indexes = np.arange(batch_size * batch, batch_size * (batch + 1))             
                # из тренировочных данных выбираем текущий батч
                current_batch_ratings = train[mixed_order[batch_indexes], :]
                # выборка случайных значений из тестового набора с размером равным размеру батча тренировочных данных
                sample_test = test[np.random.choice(num_tests, batch_size), :]
                # обновление матриц U и I 
                self.update(current_batch_ratings)
                # потери тренировочных и тестовых данных
                train_loss = self.loss(current_batch_ratings)
                test_loss  = self.loss(sample_test)
                avg_train_loss.append(train_loss)
                avg_test_loss.append(test_loss)
            
            avg_train_loss = np.mean(avg_train_loss) / float(batch_size) # средняя ошибка тренировочных данных
            avg_test_loss  = np.mean(avg_test_loss) / float(batch_size) # средняя ошибка тестовых данных
            print(f'{arrow.now()} Epoch № {self.e}', file=sys.stderr)
            print(f'{arrow.now()} Training loss:\t{avg_train_loss}', file=sys.stderr)
            print(f'{arrow.now()} Testing loss:\t{avg_test_loss}', file=sys.stderr)

In [10]:
# считываем файл csv с обработанными данными
df_review = pd.read_csv("review_data_state.csv")
# заменяем отсутствующие отзывы на пустую строку 
df_review["text"].fillna('', inplace=True)
print(df_review.head())
df_review = df_review[['user_id', 'business_id', 'stars', 'text']]
# соотвествие уникальных id пользователей числовым значениям
dict_users = {key: val for val, key in enumerate(df_review['user_id'].unique())}
inv_dict_users = {val: key for key, val in dict_users.items()}
# соотвествие уникальных id заведений числовым значениям
dict_items = {key: val for val, key in enumerate(df_review['business_id'].unique())}
inv_dict_items = {val: key for key, val in dict_items.items()}
    
df_review['user_id'] = df_review['user_id'].swifter.apply(lambda x: dict_users[x])
df_review['business_id'] = df_review['business_id'].swifter.apply(lambda x: dict_items[x])
    
# разделяем данные на тренировочные и тестовые
train_ratings_df, test_ratings_df = train_test_split(df_review, test_size = 0.2)

print(f'Size of training data: {train_ratings_df.shape}')
print(f'Size of test data: {test_ratings_df.shape}')

                review_id                 user_id             business_id  \
0  01DNCgnW8K6xSyv4krYa1w  XivuRnPq3D-K5cPiBTztIA  oyjkEn5fAqcGNsBbqc8LTw   
1  2dYcxwdtulsUY8kj4oZezw  iAQFEArbXp1WUrseOFTLMA  gfhWUixZ4MmEAC1DtTsodA   
2  MRPCm_BYEHl2T9oNZYnCbQ  oAFF_A02Ws0mwH-lkbzyMQ  l_kefVF1frmC0xRW2YkvUA   
3  ktOeQ7aup9FKFSnO_M1MOg  lq_DOs_38ctAKSzdE8gW4g  N6-dAWPTAaHqwRb94vXFCQ   
4  5n7Ye0F9mS7r25rnRfPhZw  AVxQMKJsNvyNPbCJsinvYA  srHBs-NlG0upDlambkURbA   

   stars                                               text  
0    1.0  good want several time hoping experience alway...  
1    5.0  great service food amazing reasonably priced o...  
2    2.0  pro fun atmosphere great people watching mecha...  
3    5.0  homemade lasagna great gotta try rizzo spaghet...  
4    5.0  fast friendly always make sure order correct e...  



Size of training data: (244355, 4)
Size of test data: (61089, 4)


In [11]:
# для каждого пользователя и заведения объединяем все отзывы в один абзац
def get_train_whole_text(df, name_id):
    return df[[name_id,'text']].groupby(name_id).agg({'text': ' '.join})
    
    
user_id_review_df = pd.DataFrame(index=inv_dict_users.keys(), columns=["text"])
user_id_review_df.index.name = 'user_id'
business_id_review_df= pd.DataFrame(index=inv_dict_items.keys(), columns=["text"])
business_id_review_df.index.name = 'business_id'

# таблицы хранят в себе всех пользователей и заведения,
# но только те тексты рекомендаций, которые относятся к тренировочным данным
user_id_review_df['text'] =  get_train_whole_text(train_ratings_df, 'user_id')["text"]
business_id_review_df['text'] =  get_train_whole_text(train_ratings_df, 'business_id')["text"]
user_id_review_df["text"].fillna('', inplace=True)
business_id_review_df["text"].fillna('', inplace=True)

In [12]:
# выделяем признаки (feature extraction) из отзывов посредством векторизации (TFIDF Vectorizer),
# чтобы использовать текст для факторизации матрицы 
business_id_vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, max_features=5000)
business_id_vectors = business_id_vectorizer.fit_transform(business_id_review_df['text'])

user_id_vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, max_features=5000)
user_id_vectors = user_id_vectorizer.fit_transform(user_id_review_df['text'])

# матрица признаков пользователей (users-features matrix)
U = pd.DataFrame(user_id_vectors.toarray(), index=user_id_review_df.index, columns=user_id_vectorizer.get_feature_names())
U.sort_index(inplace=True)
# матрица признаков заведений (business-features matrix)
I = pd.DataFrame(business_id_vectors.toarray(), index=business_id_review_df.index, columns=business_id_vectorizer.get_feature_names())
I.sort_index(inplace=True)

matrixF = MatrixFactorization(U = U.to_numpy(), I = I.to_numpy(), learning_rate=0.07, lamda=0.1, num_epoches=8, num_batches=1000)

In [13]:
R = pd.pivot_table(df_review, values='stars', index=['user_id'], columns=['business_id'])
R.head()

business_id,0,1,2,3,4,5,6,7,8,9,...,3603,3604,3605,3606,3607,3608,3609,3610,3611,3612
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
matrixF.fit(train_ratings_df.to_numpy()[:, 0:3], test_ratings_df.to_numpy()[:, 0:3])

2020-12-22T03:01:26.013814+03:00 Epoch № 1
2020-12-22T03:01:26.014368+03:00 Training loss:	1.6696318432422341
2020-12-22T03:01:26.014469+03:00 Testing loss:	7.58301449196062
2020-12-22T03:01:47.116048+03:00 Epoch № 2
2020-12-22T03:01:47.116264+03:00 Training loss:	0.5716894569991965
2020-12-22T03:01:47.116348+03:00 Testing loss:	5.7966760804356525
2020-12-22T03:02:07.965097+03:00 Epoch № 3
2020-12-22T03:02:07.965313+03:00 Training loss:	0.5135235352743859
2020-12-22T03:02:07.965398+03:00 Testing loss:	5.653977140957659
2020-12-22T03:02:28.452314+03:00 Epoch № 4
2020-12-22T03:02:28.452535+03:00 Training loss:	0.49919191688600045
2020-12-22T03:02:28.452632+03:00 Testing loss:	5.62603225353141
2020-12-22T03:02:51.674160+03:00 Epoch № 5
2020-12-22T03:02:51.674788+03:00 Training loss:	0.49281346167534845
2020-12-22T03:02:51.674887+03:00 Testing loss:	5.613704756769916
2020-12-22T03:03:13.471083+03:00 Epoch № 6
2020-12-22T03:03:13.471296+03:00 Training loss:	0.48893665623588156
2020-12-22T03

In [15]:
U = pd.DataFrame(matrixF.U, index=user_id_review_df.index, columns=user_id_vectorizer.get_feature_names())
U.sort_index(inplace=True)
I = pd.DataFrame(matrixF.I, index=business_id_review_df.index, columns=business_id_vectorizer.get_feature_names())
I.sort_index(inplace=True)

In [16]:
if __name__ == '__main__':
    R = np.dot(matrixF.U, matrixF.I.T)
    with Pool(processes = 10) as pool:
        R = np.array(pool.map(correct_numbers, R))

In [17]:
# предсказанные рейтинги
R = pd.DataFrame(R, index=U.index, columns=I.index)

In [18]:
print(f"Predicted ratings:\n")
R.head()

Predicted ratings:



business_id,0,1,2,3,4,5,6,7,8,9,...,3603,3604,3605,3606,3607,3608,3609,3610,3611,3612
user_id,,,,,,,,,,,,,,,,,,,,,
0,1,3,2,3,3,3,4,4,4,3,...,3,2,2,1,3,2,2,3,2,1
1,2,5,2,3,3,3,3,3,4,3,...,3,2,2,1,3,2,2,3,2,1
2,1,1,2,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,2,3,2,4,3,3,3,3,3,3,...,3,2,2,1,3,2,2,2,2,1
4,1,2,2,2,5,2,3,3,2,2,...,2,2,1,1,2,2,2,2,1,1


In [19]:
# топ-10 признаков по первому заведению
I.iloc[train_ratings_df.iat[0, 1]].sort_values(ascending=False).head(10)

oak            0.889593
steak          0.608566
steakhouse     0.416135
service        0.405721
restaurant     0.368630
filet          0.342638
great          0.338519
back           0.327996
anniversary    0.292656
time           0.288251
Name: 441, dtype: float64

In [20]:
# топ-10 признаков для первого пользователя
U.iloc[train_ratings_df.iat[0, 0]].sort_values(ascending=False).head(10)

food          0.403544
good          0.396688
great         0.391825
place         0.301927
service       0.300061
restaurant    0.276190
drink         0.257380
time          0.241350
also          0.238057
back          0.236349
Name: 12232, dtype: float64

In [21]:
# сохраняем матрицы U, I и vectorizer
data = {'U': U, 'I': I, 'V': user_id_vectorizer}

with open('recommendation_model.pkl', 'wb') as f:
     pickle.dump(data, f)

In [22]:
# записываем полученный файл без индексов
R.to_csv('predicted_ratings.csv', sep='\t')

In [23]:
# загружаем сохраненную модель
with open('recommendation_model.pkl', 'rb') as f:
     model = pickle.load(f)
print(model)

{'U':           ability      able  absolute  absolutely    absurd  abundance  \
user_id                                                                  
0        0.003075  0.015179  0.001190    0.010062  0.001064  -0.001781   
1       -0.000711  0.012330  0.019986    0.040658 -0.000300   0.003463   
2        0.000172 -0.003811 -0.000678    0.021446 -0.000082  -0.000442   
3       -0.002647  0.031522 -0.002732    0.095153 -0.000809   0.000707   
4        0.000261  0.009663  0.001342    0.013270 -0.000539   0.000288   
...           ...       ...       ...         ...       ...        ...   
100906   0.001038  0.008115  0.001972    0.007075 -0.000027   0.000227   
100907   0.008231  0.099200  0.004034    0.149505  0.001400  -0.000586   
100908  -0.000261  0.009413  0.002518    0.023936  0.000675   0.000241   
100909   0.021060  0.024797 -0.000813    0.024922  0.000540  -0.000052   
100910   0.018564  0.023705  0.005630    0.052293  0.002745  -0.000805   

         abundant        ac    

In [24]:
I = model["I"]
U = model["U"]
user_id_vectorizer = model["V"] 

In [25]:
R = pd.read_csv('predicted_ratings.csv', sep='\t')
R.head()

,user_id,0,1,2,3,4,5,6,7,8,...,3603,3604,3605,3606,3607,3608,3609,3610,3611,3612
0,0,1,3,2,3,3,3,4,4,4,...,3,2,2,1,3,2,2,3,2,1
1,1,2,5,2,3,3,3,3,3,4,...,3,2,2,1,3,2,2,3,2,1
2,2,1,1,2,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,3,2,3,2,4,3,3,3,3,3,...,3,2,2,1,3,2,2,2,2,1
4,4,1,2,2,2,5,2,3,3,2,...,2,2,1,1,2,2,2,2,1,1


In [26]:
words = input("What place are you looking for? ")
#words = "Italian dinner with beautiful views and tasty wine"


query = pd.DataFrame([text_preprocess(words)], columns=['text']) # создаем датафрейм с заданным запросом
query_vector = user_id_vectorizer.transform(query['text']) # выделение признаков запроса
query = pd.DataFrame(query_vector.toarray(), index=query.index, columns=user_id_vectorizer.get_feature_names())

predictRating=pd.DataFrame(np.dot(query.loc[0],I.T), index=I.index, columns=['rating'])
topRecommendations=pd.DataFrame.sort_values(predictRating,['rating'],ascending=[0])[:5]

number = 0
for i in topRecommendations.index:
    number+=1
    print(f"\n№ {number}")
    print(f"Name: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['name'].iloc[0]}")
    print(f"City, state: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['city'].iloc[0]}, {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['state'].iloc[0]}")
    print(f"Category: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['categories'].iloc[0]}")
    print(f"Average star: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['stars'].iloc[0]}")
    print(f"Review count: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['review_count'].iloc[0]}")

What place are you looking for? Italian dinner with beautiful views and tasty wine

№ 1
Name: Fahrenheit - Charlotte
City, state: Charlotte, NC
Category: Salad, Pizza, Restaurants, American (New)
Average star: 3.0
Review count: 794

№ 2
Name: Lake Norman Cottage
City, state: Davidson, NC
Category: Restaurants, Wine Bars, Tapas/Small Plates, Beer, Wine & Spirits, Food, Nightlife, Bars
Average star: 4.0
Review count: 30

№ 3
Name: Antico Italian Restaurant
City, state: Huntersville, NC
Category: Pizza, Restaurants, Italian, Seafood
Average star: 4.5
Review count: 183

№ 4
Name: North Harbor Club
City, state: Davidson, NC
Category: Restaurants, American (New)
Average star: 4.0
Review count: 251

№ 5
Name: Wooden Vine Wine Bar & Bistro
City, state: Charlotte, NC
Category: Restaurants, Nightlife, Bistros, Breakfast & Brunch, Wine Bars, Bars, American (New)
Average star: 3.5
Review count: 167
